In [432]:
from urllib.parse import urlparse
from urllib.parse import parse_qs
import math
from youtube_transcript_api import YouTubeTranscriptApi

In [433]:
# yt_video_list=["https://www.youtube.com/watch?v=4qh_rguqdD4",
#                "https://www.youtube.com/watch?v=rR_mlBb-5-k&t=1211s"
#                ,"https://www.youtube.com/watch?v=k-4kdiEqCYg",
#                "https://www.youtube.com/watch?v=GX4nd41Iu-Q",
#                "https://www.youtube.com/watch?v=L65CxuCt-bg",
#                "https://www.youtube.com/watch?v=hSXlv26f_Kg&t=783s",
#                "https://www.youtube.com/watch?v=BYkPDg7RZa4",
#                "https://www.youtube.com/watch?v=_2k1pHKSHtg",
#                "https://www.youtube.com/watch?v=ibQn5wau_VA",
#                "https://www.youtube.com/watch?v=Jie6URfi1HY"]
yt_video_list=["https://www.youtube.com/watch?v=4qh_rguqdD4",
               "https://www.youtube.com/watch?v=rR_mlBb-5-k&t=1211s"]

In [434]:
list_of_chunks_all_videos=[]    
for video_url in yt_video_list:
    list_of_lines=[]  #for current video
    url_data=urlparse(video_url)
    query = parse_qs(url_data.query)
    video_id = query["v"][0]
    print(video_id)
    transcritps = YouTubeTranscriptApi.get_transcript(video_id)
    for i in transcritps:
        time=i['start']
        time=math.floor(time)
        hours=int(time//3600)
        minutes=int((time%3600)//60)
        secs=time%60
        # print( f"{hours:02}:{minutes:02}:{secs:06.3f}")
        i['start']=f"{hours:02}:{minutes:02}:{secs:06.3f}"
        ## and adding youtube video
        i['video_url']=video_url   # adding youtube video url
        list_of_lines.append({'time':i['start'],'text':i['text'],'video_url':i['video_url']})
    list_of_chunks=[]
    chunk=[]
    chunk.append(list_of_lines[0])
    chunk.append(list_of_lines[1])
    no=0
    for i in list_of_lines[2:]:
        chunk.append(i)
        no+=1
        if len(chunk)>8:
            chunk.pop(0)
        if (no)%6==0:
            chunk_start=chunk[0]['time']
            chunk_url=chunk[0]['video_url']
            chunk_text=''
            count=0
            for j in chunk:
                count+=1
                chunk_text+=j['text']
                chunk_text+=' '
            # print(count)
            # print(chunk_text)
            # print()
            list_of_chunks.append({'time':chunk_start,'text':chunk_text,'video_url':video_url})
            no=0
    if len(chunk)!=0:
        chunk_start=chunk[0]['time']
        chunk_url=chunk[0]['video_url']
        chunk_text=''
        count=0
        for j in chunk:
            count+=1
            chunk_text+=j['text']
            chunk_text+=' '
        # print(count)
        # print(chunk_text)
        # print()
        list_of_chunks.append({'time':chunk_start,'text':chunk_text,'video_url':video_url})
    no=0
    print(len(list_of_chunks))
    list_of_chunks_all_videos.extend(list_of_chunks)
    print("the len now is ",len(list_of_chunks_all_videos))
    

4qh_rguqdD4
101
the len now is  101
rR_mlBb-5-k
99
the len now is  200


In [435]:
print(list_of_chunks_all_videos)

[{'time': '00:00:00.000', 'text': "e [Music] [Music] hello hello and happy Monday it's a May bank holiday at least in the UK I hope it you're having a wonderful Monday either way wherever you are if you're not having a bank holiday my name is ", 'video_url': 'https://www.youtube.com/watch?v=4qh_rguqdD4'}, {'time': '00:00:50.000', 'text': "either way wherever you are if you're not having a bank holiday my name is Mandy Seline I'm a director and DP and I'm the founder of breadbox Studios which is a boutique post and production house kind of based in the UK we specialize in branded and digital first content in general so we work with a lot ", 'video_url': 'https://www.youtube.com/watch?v=4qh_rguqdD4'}, {'time': '00:01:01.000', 'text': "specialize in branded and digital first content in general so we work with a lot of creators as well for those of you who don't know the last few Mondays of May I've been hosting these challenges every Monday and it's the last Monday of May so this is my la

In [436]:
# put into a df and vectorizing

In [437]:
import pandas as pd

In [438]:
df=pd.DataFrame(list_of_chunks_all_videos)

In [439]:
df.shape

(200, 3)

In [440]:
# df.head(400)

In [441]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings("ignore", message="Unverified HTTPS request")
warnings.filterwarnings("ignore", message="InsecureRequestWarning")

In [442]:
from opensearchpy import OpenSearch

CLUSTER_URL = 'https://localhost:9200'

def get_os_client(cluster_url = CLUSTER_URL,
                  username='admin',
                  password='admin'):
    '''
    Get OpenSearch client
    :param cluster_url: cluster URL like https://ml-te-netwo-1s12ba42br23v-ff1736fa7db98ff2.elb.us-west-2.amazonaws.com:443
    :return: OpenSearch client
    '''
    client = OpenSearch(
        hosts=[cluster_url],
        http_auth=(username, password),
        verify_certs=False
    )
    return client

client = get_os_client()

/Users/vpatwal/Desktop/trying_summarization/.venv/lib/python3.10/site-packages/opensearchpy/connection/http_urllib3.py:208: UserWarning: Connecting to https://localhost:9200 using SSL with verify_certs=False is insecure.
  warnings.warn(


In [443]:
from sentence_transformers import SentenceTransformer

MODEL_NAME = "msmarco-distilbert-base-tas-b"


In [444]:
model = SentenceTransformer(MODEL_NAME)

In [445]:
EMBEDDING_DIM = model.encode(["Sample sentence"])[0].shape[0]
print(EMBEDDING_DIM)#will be used ahead

768


In [562]:
index_name = "new_one"

index_body = {
  "settings": {
    "index": {
      "knn": True,
      "knn.algo_param.ef_search": 100
    }
  },
  "mappings": { #how do we store, 
    "properties": {
        "embedding": {
          "type": "knn_vector", #we are going to put 
          "dimension": EMBEDDING_DIM,
          "method": {
            "name": "hnsw",
            "space_type": "l2",
            "engine": "faiss",
            "parameters": {
              "ef_construction": 128,
              "m": 24
            }
         }
     }
    }
}}


In [563]:
response = client.indices.delete(index = index_name)

In [564]:
response = client.indices.create(index=index_name, body=index_body)

In [565]:
df.head()
df.columns

Index(['time', 'text', 'video_url'], dtype='object')

In [566]:
import pandas as pd

for idno, row in df.iterrows():
    print(f"Id: {idno}, Start: {row['time']}, Overview: {row['text']},url:{row['video_url']}")
    # original_title = row['time']
    overview = row['text']
    id = row['time']
    st_time=row['time']
    url=row['video_url']
    # Sentence transformer model takes list of documents as input and returns list of embeddings.
    embedding = model.encode([overview])[0]
    # We are inserting a data point with 3 attribute, "id", "text" and "embedding" as knn_vector type.
    my_doc = {"id": str(idno), "plot": overview, "embedding": embedding,"time" : st_time,"url":url}
    res = client.index(
        index=index_name,
        body=my_doc,
        id = str(idno),
        refresh = True
        )

Id: 0, Start: 00:00:00.000, Overview: e [Music] [Music] hello hello and happy Monday it's a May bank holiday at least in the UK I hope it you're having a wonderful Monday either way wherever you are if you're not having a bank holiday my name is ,url:https://www.youtube.com/watch?v=4qh_rguqdD4
Id: 1, Start: 00:00:50.000, Overview: either way wherever you are if you're not having a bank holiday my name is Mandy Seline I'm a director and DP and I'm the founder of breadbox Studios which is a boutique post and production house kind of based in the UK we specialize in branded and digital first content in general so we work with a lot ,url:https://www.youtube.com/watch?v=4qh_rguqdD4
Id: 2, Start: 00:01:01.000, Overview: specialize in branded and digital first content in general so we work with a lot of creators as well for those of you who don't know the last few Mondays of May I've been hosting these challenges every Monday and it's the last Monday of May so this is my last one sad face but

In [567]:
# "query": {
#     "hybrid": {
#         "queries": [
#         {
#             "match": {
#             "passage_text": {
#                 "query": "Hi world"
#             }
#             }
#         },
#         {
#             "neural": {
#             "passage_embedding": {
#                 "query_text": "Hi world",
#                 "model_id": "aVeif4oB5Vm0Tdw8zYO2",
#                 "k": 5
#             }
#             }
#         }
#         ]
#     }
# }

In [580]:
""" Example query text """
# user_query = "Applying Lens Blur"
# user_query = "hide content"
# user_query = "expanding"
# user_query="deleting people"
# user_query='How does the Generative AI Remove feature in Lightroom work?'
# user_query="What steps does Jacob describe for creating a drop shadow in Illustrator?"
# user_query="Inserting drop shadow"
# user_query="cut it"
# user_query="How to create a montage using markers in Premiere Pro?"
# user_query="select and arrange photos for a video montage?"
# user_query="add markers to a timeline for precise editing"
# user_query="delete unwanted sections of a video sequence in Premiere Pro"
# user_query="Asset Availability and Customization"
# user_query="Asset availability"
user_query="download assets"


""" Embedding the query by using the same model """
query_embedding = model.encode((user_query))


query_body = { #most initial one
    "query": {"knn": {"embedding": {"vector": query_embedding, "k": 10}}},
    "_source": False,
    "fields": ["id", "title", "plot","time","url"],
    "size":10  # vivek added this , top 10
}
query_body = {  # works but its kinda post filtering
    #In this scenario, we can say that the URL matching is not really a pre-filtering or post-filtering step. Instead, it's more like a conjunctive filtering step, where both the URL match and KNN search conditions must be true for a document to be considered a match.

# The URL match is not filtering the documents before the KNN search,
#  nor is it filtering the results after the KNN search. Instead,
#    it's a condition that's applied in parallel with the KNN search, 
#    and both conditions must be satisfied for a document to be included in the results.

# So, in this case, we can't really say it's pre-filtering or post-filtering,
#  but rather a conjunctive filtering approach.


    # "from":15,  
    # "size":15,
    "query": {
        "bool": {
            "must": [
                {
                    "match": {
                        "url": {
                            "query": "https://www.youtube.com/watch?v=rR_mlBb-5-k&t=1211s",
                            # "query":"https://www.youtube.com/watch?v=4qh_rguqdD4",
                            # "boost": 1000  # Adjust boost factor for this specific URL match
                        }
                    }
                },
                {
                    "knn": {
                        "embedding": {
                            "vector": query_embedding,
                            "k": 50
                        }
                    }
                }
            ]
        }
    },
    "_source": False,
    "fields": ["id", "title", "plot", "time", "url"],
    "size": 200
}
# query_body={
#   "query": {
#     "bool": {
#       "should": [
#         {
#           "match": {
#             "url": {
#               "query": "https://www.youtube.com/watch?v=4qh_rguqdD4",
#               "boost": 2
#             }
#           }
#         },
#         {
#           "knn": {
#             "embedding": {
#               "vector": query_embedding,
#               "k": 20
#             }
#           }
#         }
#       ],
#       "minimum_should_match": 1
#     }
#   },
#   "_source": False,
#   "fields": ["id", "title", "plot", "time", "url"],
#   "size": 100
# }

# query_body = {
#     "query": {
#         "bool": {
#             "filter": [
#                 {
#                     "match": {
#                         "url": {
#                             # "query": "https://www.youtube.com/watch?v=rR_mlBb-5-k&t=1211s",
#                             "query":"https://www.youtube.com/watch?v=4qh_rguqdD4",
#                             "boost": 1  # Adjust boost factor for this specific URL match
#                         }
#                     }
#                 }
#                 ],
#                 "should":
#                 [
#                     {
#                     "knn": {
#                         "embedding": {
#                             "vector": query_embedding,
#                             "k": 100
#                         }
#                     }
#                     }
#                 ]
#         }
#     },
#     "_source": False,
#     "fields": ["id", "title", "plot", "time", "url"],
#     "size": 10
# }



# query_body = {
#   "query": {
#     "script_score": {
#       "query": {
#         "bool": {
#           "filter": [
#             {
#               "term": {
#                 "url": ""
#               }
#             }
#           ]
#         }
#       },
#       "script": {
#         "source": "knn_score",
#         "lang": "knn",
#         "params": {
#           "field": "embedding",  # Assuming your k-NN field is named "embedding"
#           "query_value": query_embedding,
#           "space_type": "l2"
#         }
#       }
#     }
#   },
# #   "_source": False,
#   "fields": ["id", "title", "plot", "time", "url"],
#   "size": 10
# }


results = client.search(
    body=query_body,
    index=index_name
)



## must and should be's are similar but lets say i took k=10, and if we have it , then if it doesnt match it can still be used 
## i think should is more strict, like in must it runs both and then weighs
## eg for k =10 and size =100

In [581]:
print(query_embedding)
type(query_embedding)
query_embedding.shape

[ 3.09841745e-02 -5.06544471e-01 -2.66774416e-01  1.53043941e-01
  2.13237509e-01  1.76238529e-02  1.02386951e-01 -5.83936311e-02
  1.84178144e-01  1.00112647e-01 -5.08252323e-01  3.98085684e-01
 -2.41592303e-01  2.17623711e-01 -3.83082367e-02 -2.22525164e-01
 -4.74569738e-01  3.85512888e-01  1.92874074e-01 -1.95207307e-03
 -2.98409790e-01  1.57247826e-01 -1.89679578e-01 -1.87526181e-01
  2.31971592e-01  5.75771146e-02  8.64489749e-02 -2.52956539e-01
 -4.57088411e-01 -8.02440569e-03 -4.63834167e-01 -1.21691793e-01
  4.46024418e-01  1.21511444e-01 -1.79475576e-01  1.23782888e-01
 -4.19466853e-01  4.95083556e-02  6.33346289e-03 -1.20440006e-01
  4.61334884e-01  1.72480494e-01  1.25618890e-01  2.80813456e-01
 -2.70205438e-02  1.26755357e-01  4.67200503e-02  1.05050415e-01
 -6.69453144e-02 -3.96326661e-01 -2.95355022e-02  1.51652858e-01
  3.37027580e-01 -4.34554935e-01  4.17589873e-01  4.38633233e-01
  3.90381441e-02 -1.30867198e-01 -7.27038831e-03  1.72313333e-01
  4.08969551e-01  1.87080

(768,)

In [582]:
print(results)

{'took': 39, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 100, 'relation': 'eq'}, 'max_score': 3.1704648, 'hits': [{'_index': 'new_one', '_id': '177', '_score': 3.1704648, 'fields': {'id': ['177'], 'time': ['00:19:38.000'], 'plot': ["downloads let's also do one more just for the sake of variants this guy right over here perfect that brought that over to our library saved it to our downloads let's go back to our illustrator program and let's import these stock images "], 'url': ['https://www.youtube.com/watch?v=rR_mlBb-5-k&t=1211s']}}, {'_index': 'new_one', '_id': '176', '_score': 3.1658828, 'fields': {'id': ['176'], 'time': ['00:19:26.000'], 'plot': ["scroll back up it's not overom complicated let's go with this guy lots of options very colorful let's go ahead and license that notice it saved to my library and it's saving to my downloads let's also do one more just for the sake of variants this guy right "], 'url':

In [583]:
for i, result in enumerate(results["hits"]["hits"]):
    # print(i,results)
    plot = result['fields']['plot'][0]
    url = result['fields']['url'][0]
    score = result['_score']
    time=result['fields']['time'][0]
    id=result["_id"]
    print(f"{id}., Start Time: {time} Score: {score}, Url: {url}  Plot: {plot}")
# for i, result in enumerate(results["hits"]["hits"]):
#     # print(i,results)
#     plot = result['_source']['plot']
#     url = result['_source']['url']
#     # score = result['_score']
#     # time=result['fields']['time'][0]
#     # id=result["_id"]
#     # print(f"{id}., Start Time: {time} Score: {score}, Url: {url}  Plot: {plot}")
#     # print(result['_source'].keys())
#     print(url)
#     print(plot)


177., Start Time: 00:19:38.000 Score: 3.1704648, Url: https://www.youtube.com/watch?v=rR_mlBb-5-k&t=1211s  Plot: downloads let's also do one more just for the sake of variants this guy right over here perfect that brought that over to our library saved it to our downloads let's go back to our illustrator program and let's import these stock images 
176., Start Time: 00:19:26.000 Score: 3.1658828, Url: https://www.youtube.com/watch?v=rR_mlBb-5-k&t=1211s  Plot: scroll back up it's not overom complicated let's go with this guy lots of options very colorful let's go ahead and license that notice it saved to my library and it's saving to my downloads let's also do one more just for the sake of variants this guy right 
181., Start Time: 00:20:42.000 Score: 3.1650138, Url: https://www.youtube.com/watch?v=rR_mlBb-5-k&t=1211s  Plot: edit should be a crop option somewhere here I might be looking right over it I'm trying to speed a little bit let's see select that's okay we can work around this f

In [584]:
len(results["hits"]['hits'])

100

In [585]:
print(results["hits"]["hits"])

[{'_index': 'new_one', '_id': '177', '_score': 3.1704648, 'fields': {'id': ['177'], 'time': ['00:19:38.000'], 'plot': ["downloads let's also do one more just for the sake of variants this guy right over here perfect that brought that over to our library saved it to our downloads let's go back to our illustrator program and let's import these stock images "], 'url': ['https://www.youtube.com/watch?v=rR_mlBb-5-k&t=1211s']}}, {'_index': 'new_one', '_id': '176', '_score': 3.1658828, 'fields': {'id': ['176'], 'time': ['00:19:26.000'], 'plot': ["scroll back up it's not overom complicated let's go with this guy lots of options very colorful let's go ahead and license that notice it saved to my library and it's saving to my downloads let's also do one more just for the sake of variants this guy right "], 'url': ['https://www.youtube.com/watch?v=rR_mlBb-5-k&t=1211s']}}, {'_index': 'new_one', '_id': '181', '_score': 3.1650138, 'fields': {'id': ['181'], 'time': ['00:20:42.000'], 'plot': ["edit sh

In [534]:
# client({from=0&size=10})

In [472]:
df.to_csv("table3")